In [9]:
import torch
import torchtext
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import jieba
import pandas as pd
import re
from tqdm.notebook import tqdm
from collections import Counter

# 1.准备数据 

In [2]:
# 构造数据集
class MyDataset(Dataset):
    def __init__(self, file_path, tokenizer, stopwords, debug=True):
        df = pd.read_csv(file_path)
        df = df.dropna().reset_index(drop=True)
        if debug:
            df = df.sample(2000).reset_index(drop=True)
        else:
            df = df.sample(50000).reset_index(drop=True)
        # 读取常用停用词
        counter = Counter()
        sentences = []
        for title in tqdm(df['title']):   
            # 去除标点符号
            title = re.sub(r'[^\u4e00-\u9fa5]', '', title)
            tokens = [token for token in tokenizer(title.strip()) if token not in stopwords]
            counter.update(tokens)
            sentences.append(tokens)
        self.vocab = torchtext.vocab.vocab(counter, specials=['<unk>', '<pad>'])
        
        # 构造输入和输出，输入是每三个词，输出是这三个词的下一个词，也就是简单的n-gram语言模型（n=3）
        self.inputs = []
        self.labels = []
        for sen in sentences:
            for i in range(len(sen) - 3):
                self.inputs.append(self.vocab.lookup_indices(sen[i: i + 3]))
                self.labels.append([self.vocab[sen[i + 3]]])

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # 返回一个x和一个y
        return torch.LongTensor(self.inputs[idx]), torch.LongTensor(self.labels[idx])

In [3]:
file_path = '../data/THUCNews/train.csv'
debug = True
tokenizer = torchtext.data.utils.get_tokenizer('spacy', language='zh_core_web_sm')
stopwords = [line.strip() for line in open('../stopwords/cn_stopwords.txt', 'r', encoding='utf-8').readlines()]
dataset = MyDataset(file_path, tokenizer, stopwords)
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

  0%|          | 0/2000 [00:00<?, ?it/s]

# 2.构建模型

In [4]:
class NNLM(nn.Module):
    def __init__(self, vocab_size, embed_size, n_step, n_hidden):
        super().__init__()
        self.embed_size = embed_size
        self.n_step = n_step
        # vocab size投影到到embed size的空间中
        self.embedding = nn.Embedding(vocab_size, embed_size)
        # 构造一个隐藏层，输入大小为 步长 * embed size，输入大小为n_hidden
        self.linear = nn.Linear(n_step * embed_size, n_hidden)
        # 将n_hidden投影回vocab size大小
        self.output = nn.Linear(n_hidden, vocab_size)

    def forward(self, X):
        X = self.embedding(X)
        X = X.view(-1, self.n_step * self.embed_size)
        X = self.linear(X)
        X = torch.tanh(X)
        y = self.output(X)
        return y

In [5]:
# 初始化模型
model = NNLM(vocab_size=len(dataset.vocab), embed_size=256, n_step=3, n_hidden=256)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 查看模型
print(model)

NNLM(
  (embedding): Embedding(8358, 256)
  (linear): Linear(in_features=768, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=8358, bias=True)
)


# 3.训练模型

In [6]:
# 训练20个epoch
for epoch in range(20):
    for train_input, train_label in dataloader:
        output = model(train_input)
        loss = criterion(output, train_label.squeeze_())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('epoch:', epoch + 1, 'loss =', '{:.6f}'.format(loss))

epoch: 1 loss = 9.084500
epoch: 2 loss = 7.712342
epoch: 3 loss = 6.277411
epoch: 4 loss = 4.700910
epoch: 5 loss = 3.170568
epoch: 6 loss = 1.630075
epoch: 7 loss = 0.590093
epoch: 8 loss = 0.287548
epoch: 9 loss = 0.189612
epoch: 10 loss = 0.099930
epoch: 11 loss = 0.081222
epoch: 12 loss = 0.066866
epoch: 13 loss = 0.052047
epoch: 14 loss = 0.042389
epoch: 15 loss = 0.037109
epoch: 16 loss = 0.029919
epoch: 17 loss = 0.028010
epoch: 18 loss = 0.023675
epoch: 19 loss = 0.022178
epoch: 20 loss = 0.018222


# 4.预测

In [10]:
# 使用训练好的模型进行预测，train_input直接是上面代码中的，直接用
# 模型输出之后取argmax，再用idx2token转回单词，查看效果，可以看到效果还可以，有上下文关系
predict = model(train_input).data.max(1, keepdim=True)[1].squeeze_().tolist()
input_list = train_input.tolist()
for i in range(len(input_list)):
    print(dataset.vocab.get_itos()[input_list[i][0]] + ' ' +  
          dataset.vocab.get_itos()[input_list[i][1]] + ' ' + 
          dataset.vocab.get_itos()[input_list[i][2]] + ' -> ' + 
          dataset.vocab.get_itos()[predict[i]])

市场 跷 跷板 -> 效应
膝伤 欧冠 恐缺 -> 战马赛
巴顿 红 牛距 -> 数
塞班诺基亚 推亿 塞班 -> 手机
危机 冲击 传统 -> 教材
年 河北 秦皇岛 -> 中考
创业板 上市 门槛 -> 提高
皇马 巴萨大乱 战 -> 前因后果
突破 转播 北美 -> 落地
阿布 亲临 前线 -> 留特里德罗巴
项目 涉及 金额 -> 超亿
原 高管 退位 -> 兵装摩
庆生 王心 凌送 -> 巨乳
夫妇 称忠于 良知 -> 代母
刺激 资金 进一步 -> 回流
仅 需元 东芝 -> 英寸
吴雨霏 邓丽欣 再次 -> 合作
消息 利好 美股 -> 走强
洁欢 乐购 佛山站 -> 盛大
几率 达 五成 -> 排除
许志安 新年 断 -> 懒根
季节 交替 易 -> 敏春季
改革 法案 争议 -> 中
赵柯 写 真照 -> 曝光
切尔西盯 防梅西 候选 -> 曝光
次 尿检 大麻 -> 案件
发生 里 氏级 -> 地震
排名 揭晓 王孙 -> 战
宾 新 换镜 -> 谍报
组织 慈善 赛格 -> 里
梁痴人 说 梦末 -> 位龙
凯特 温丝莱 特甩 -> 花心
周欢 乐派 献饕 -> 餮盛
历时 月 自费 -> 拍摄
垂涎悍 复出 带来 -> 鲶鱼
模式 机奥 南宁 -> 仅
波拒 洋帅 老 -> 领导
涨 停敢 死队 -> 火线
逃亡 香格 里 -> 拉开
快讯 阿里巴巴财年 净利 -> 亿
乡村 基周三 午盘 -> 转涨
令 连续 错过 -> 大牌
城 综合 评价 -> 组图
冯绍峰 杨幂 牵手 -> 荣膺
复古 作 高像素 -> 富士
要求 美军 停止 -> 空袭
状告 法院 案孝感 -> 中院
秘诀 裙子 一定 -> 迷你
下挫 八大 机构 -> 后市
世锦赛 征战 史梦 -> 队
星巴克 供应 商塔塔 -> 咖啡拟
基金 大佬 变动潮 -> 重现
喝 活 周 -> 揭
双模 轻松 上网 -> 天语
委内瑞拉 有意 探索 -> 新
东方 广场 五星级 -> 写字楼
欧青赛 英格兰 逆转 -> 西班牙
销量 数字 称 -> 遭
接受 暧昧 尺度 -> 图
绿色 源料 项目 -> 成功
诈骗 实录 通讯 -> 网络
陈冠希 冒死 亮相 -> 记者会
转会 广厦 陈照升 -> 加盟
北京 考门 实践 -> 课
定位 商务 便携 -> 本售
公证 妻子 引发 -> 